In [1]:
import tkinter as tk
from tkinter import messagebox
import random
import pygame

# Function to sum three values
def sum(a, b, c):
    return a + b + c

# Function to check if there's a winning combination in the current game state
def checkWin(state):
    # All possible winning combinations (indices of the board)
    wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]
    
    for win in wins:
        if sum(state[win[0]], state[win[1]], state[win[2]]) == 3:
            return True, win  # Return True and the winning combination if a win is detected
    return False, []  # Return False and an empty list if no win is detected

# Function to play a sound when there's a win or a tie
def play_sound():
    pygame.mixer.music.load("win_sound.mp3")  # Load the win sound file
    pygame.mixer.music.play()  # Play the sound

# Function to handle the click event on the board buttons
def click(event):
    global turn, single_player
    btn = event.widget
    idx = int(btn.grid_info()['row'] * 3 + btn.grid_info()['column'])  # Get the index of the button clicked
    
    if xstate[idx] == 0 and zstate[idx] == 0:  # Check if the cell is empty
        if turn == 'X':
            xstate[idx] = 1
            btn.config(text='X', state=tk.DISABLED)  # Mark the button with 'X' and disable it
            turn = 'O'
        elif not single_player:
            zstate[idx] = 1
            btn.config(text='O', state=tk.DISABLED)  # Mark the button with 'O' and disable it
            turn = 'X'

        # Check for a winner
        winner, win_combo = checkWin(xstate) if turn == 'O' else checkWin(zstate)
        if winner:
            play_sound()
            for i in win_combo:
                buttons[i].config(bg='lightgreen')  # Highlight the winning combination
            draw_line(win_combo)  # Draw the winning line
            messagebox.showinfo("Tic-Tac-Toe", f"{'X' if turn == 'O' else 'O'} wins!")  # Show the win message
            reset_board()  # Reset the board
        elif all(xstate[i] == 1 or zstate[i] == 1 for i in range(9)):
            play_sound()
            messagebox.showinfo("Tic-Tac-Toe", "It's a tie!")  # Show the tie message
            reset_board()  # Reset the board
        elif single_player and turn == 'O':
            root.after(500, bot_move)  # Make the bot move after a short delay

# Function for the bot's move in single player mode
def bot_move():
    global turn
    move = find_best_move()  # Find the best move for the bot
    if move is not None:
        zstate[move] = 1
        buttons[move].config(text='O', state=tk.DISABLED)  # Mark the bot's move and disable the button
        turn = 'X'
        
        # Check for a winner after the bot's move
        winner, win_combo = checkWin(zstate)
        if winner:
            play_sound()
            for i in win_combo:
                buttons[i].config(bg='lightgreen')  # Highlight the winning combination
            draw_line(win_combo)  # Draw the winning line
            messagebox.showinfo("Tic-Tac-Toe", "O wins!")  # Show the win message
            reset_board()  # Reset the board
        elif all(xstate[i] == 1 or zstate[i] == 1 for i in range(9)):
            play_sound()
            messagebox.showinfo("Tic-Tac-Toe", "It's a tie!")  # Show the tie message
            reset_board()  # Reset the board

# Function to find the best move for the bot
def find_best_move():
    # Check for a winning move for the bot
    for i in range(9):
        if zstate[i] == 0 and xstate[i] == 0:
            zstate[i] = 1
            if checkWin(zstate)[0]:
                return i
            zstate[i] = 0
    # Check for a blocking move against the player
    for i in range(9):
        if xstate[i] == 0 and zstate[i] == 0:
            xstate[i] = 1
            if checkWin(xstate)[0]:
                xstate[i] = 0
                return i
            xstate[i] = 0
    # Take the center if available
    if xstate[4] == 0 and zstate[4] == 0:
        return 4
    # Take any available corner
    corners = [0, 2, 6, 8]
    for corner in corners:
        if xstate[corner] == 0 and zstate[corner] == 0:
            return corner
    # Take any available side
    sides = [1, 3, 5, 7]
    for side in sides:
        if xstate[side] == 0 and zstate[side] == 0:
            return side
    # Take any remaining available move
    available_moves = [i for i in range(9) if xstate[i] == 0 and zstate[i] == 0]
    return random.choice(available_moves) if available_moves else None

# Function to draw the winning line
def draw_line(win_combo):
    coords = [
        [(50, 50), (250, 50)],  # Top row
        [(50, 150), (250, 150)],  # Middle row
        [(50, 250), (250, 250)],  # Bottom row
        [(50, 50), (50, 250)],  # Left column
        [(150, 50), (150, 250)],  # Middle column
        [(250, 50), (250, 250)],  # Right column
        [(50, 50), (250, 250)],  # Diagonal from top-left to bottom-right
        [(250, 50), (50, 250)]  # Diagonal from top-right to bottom-left
    ]
    canvas.create_line(*coords[wins.index(win_combo)], width=5, fill='red')  # Draw the line on the canvas

# Function to reset the game board
def reset_board():
    global xstate, zstate, turn
    xstate = [0] * 9
    zstate = [0] * 9
    turn = 'X'
    canvas.delete('all')  # Clear the canvas
    for btn in buttons:
        btn.config(text='', state=tk.NORMAL, bg='SystemButtonFace')  # Reset all buttons

# Function to set the game mode (single or two player)
def set_mode(mode):
    global single_player
    single_player = mode == "single"
    mode_frame.grid_forget()  # Hide the mode selection frame
    board_frame.grid(row=0, column=0)  # Show the game board frame

# Initialize game state
xstate = [0] * 9  # Player X state
zstate = [0] * 9  # Player O (bot) state
turn = 'X'  # Current turn
single_player = False  # Single player mode flag

# Winning combinations
wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]

# Initialize Pygame mixer for sound
pygame.mixer.init()

# Create main window
root = tk.Tk()
root.title("Tic-Tac-Toe")

# Create mode selection frame
mode_frame = tk.Frame(root)
mode_frame.grid(row=0, column=0)
tk.Label(mode_frame, text="Choose Mode", font=('Helvetica', 20)).grid(row=0, column=0, columnspan=2)
tk.Button(mode_frame, text="Two Player", font=('Helvetica', 14), command=lambda: set_mode("two")).grid(row=1, column=0)
tk.Button(mode_frame, text="Single Player", font=('Helvetica', 14), command=lambda: set_mode("single")).grid(row=1, column=1)

# Create game board frame
board_frame = tk.Frame(root)

# Create Canvas for drawing win line
canvas = tk.Canvas(board_frame, width=300, height=300)
canvas.grid(row=0, column=0, rowspan=3, columnspan=3)

# Create buttons for the game board and add them to the grid
buttons = []
for i in range(9):
    btn = tk.Button(board_frame, text='', width=10, height=3, font=('Helvetica', 20))
    btn.grid(row=i//3, column=i%3)
    btn.bind('<Button-1>', click)  # Bind the click event to the button
    buttons.append(btn)

# Create a reset button to reset the game board
reset_button = tk.Button(board_frame, text='Reset', width=10, height=2, font=('Helvetica', 14), command=reset_board)
reset_button.grid(row=3, column=0, columnspan=3)

# Start the main event loop
root.mainloop()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\LENOVO\.conda\envs\python_eda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16832\3102408894.py", line 51, in click
    play_sound()
  File "C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16832\3102408894.py", line 22, in play_sound
    pygame.mixer.music.load("win_sound.mp3")  # Load the win sound file
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pygame.error: No file 'win_sound.mp3' found in working directory 'c:\Users\LENOVO\AppData\Local\Programs\Microsoft VS Code'.
